In [121]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [142]:
import ogb 

import torch 
import dgl
import pandas as pd

from run import RunModel

In [207]:
name = 'cora'
name = 'citeseer'
# name = 'pubmed'
dataset = dgl.data.CitationGraphDataset(name)

X = pd.DataFrame(dataset.features.numpy())
y = pd.DataFrame(dataset.labels, columns=['class'])
networkx_graph = dataset.graph

train_mask, val_mask, test_mask = dataset.train_mask, dataset.val_mask, dataset.test_mask

task = 'classification'
metric_name = 'accuracy'

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 3327
  NumEdges: 9228
  NumFeats: 3703
  NumClasses: 6
  NumTrainingSamples: 120
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


/mnt/nfs/home/s.ivanov/envs/default/lib/python3.6/site-packages/dgl/data/citation_graph.py:258: RuntimeWarning:

divide by zero encountered in power



In [214]:
run = RunModel()
run.read_input('datasets/house_class/')
networkx_graph, X, y = run.networkx_graph, run.X, run.y

In [215]:
masks = run.masks['0']
train_mask, val_mask, test_mask = masks['train'], masks['val'], masks['test']
# task = 'regression'
# metric_name = 'loss'
task = 'classification'
metric_name = 'accuracy'

In [217]:
import models.GNN 
import models.Base
import sys
import importlib 
importlib.reload(sys.modules['models.Base'])
importlib.reload(sys.modules['models.GNN']) # these imports are necessary only for local changes to the file
from models.GNN import GNN

gnn_model = GNN(task, use_leaderboard=False)
# metrics = gnn_model.fit(run.networkx_graph, run.X, run.y, train_mask, val_mask, test_mask, num_epochs=1000, metric_name=metric_name)
metrics = gnn_model.fit(networkx_graph, X, y, train_mask, val_mask, test_mask, num_epochs=1000, metric_name=metric_name, normalize_features=True)
gnn_model.plot_interactive(metrics, legend=['train', 'val', 'test'], title='GNN model', metric_name=[metric_name, 'loss'])

Epoch 00999 | Loss 0.935 | Loss 0.617/0.615/0.604 | Time 0.0152: 100%|██████████| 1000/1000 [00:15<00:00, 63.80it/s]


Best accuracy at iteration 828: 0.610/0.616/0.600


In [218]:
import models.MLP 
import models.Base
import sys
import importlib 
importlib.reload(sys.modules['models.Base'])
importlib.reload(sys.modules['models.MLP']) # these imports are necessary only for local changes to the file
from models.MLP import MLP

gnn_model.eval()
with torch.no_grad():
    logit_t = gnn_model.model(gnn_model.graph, gnn_model.node_features).detach()
    
logit_t = None

metric_names = [metric_name, 'loss']
if logit_t is not None:
    metric_names += 'dl_loss'

mlp_model = MLP(task, dropout=0.5)
# metrics = mlp_model.fit(run.X, run.y, train_mask, val_mask, test_mask, num_epochs=1000, patience=1000, metric_name=metric_name, logit_t=logit_t)
metrics = mlp_model.fit(X, y, train_mask, val_mask, test_mask, num_epochs=1000, patience=2000, metric_name=metric_name, logit_t=logit_t, normalize_features=True)
mlp_model.plot_interactive(metrics, legend=['train', 'val', 'test'], title=f'{name} MLP', metric_name=metric_names)

Epoch 00999 | Loss 1.053 | Loss 0.493/0.479/0.492 | Time 0.0259: 100%|██████████| 1000/1000 [00:26<00:00, 37.87it/s]


Best accuracy at iteration 416: 0.535/0.541/0.545


In [219]:
import models.MLP 
import models.Base
import sys
import importlib 
importlib.reload(sys.modules['models.Base'])
importlib.reload(sys.modules['models.MLP']) # these imports are necessary only for local changes to the file
from models.MLP import MLP

gnn_model.eval()
with torch.no_grad():
    logit_t = gnn_model.model(gnn_model.graph, gnn_model.node_features).detach()
    
# logit_t = None

metric_names = [metric_name, 'loss']
if logit_t is not None:
    metric_names += ['dl_loss']

mlp_model = MLP(task, dropout=0.5)
# metrics = mlp_model.fit(run.X, run.y, train_mask, val_mask, test_mask, num_epochs=1000, patience=1000, metric_name=metric_name, logit_t=logit_t)
metrics = mlp_model.fit(X, y, train_mask, val_mask, test_mask, num_epochs=10000, patience=500, metric_name=metric_name, logit_t=logit_t, normalize_features=True)
mlp_model.plot_interactive(metrics, legend=['train', 'val', 'test'], title=f'{name} MLP', metric_name=metric_names)

Epoch 01147 | Loss 1.773 | Loss 0.523/0.512/0.531 | Time 0.0280:  11%|█▏        | 1147/10000 [00:32<04:12, 35.01it/s]


Best accuracy at iteration 646: 0.532/0.542/0.542


In [40]:
from distiller_zoo import DistillKL
import torch.nn as nn

criterion_cls = nn.CrossEntropyLoss()
criterion_div = DistillKL(4)

In [64]:
teacher_logits = gnn_model.model(gnn_model.graph, gnn_model.node_features)

In [66]:
student_logits = mlp_model.model(mlp_model.X)

In [74]:
criterion_div(student_logits, teacher_logits).detach()

tensor(2.2511, device='cuda:0')